In [ ]:
import numpy as np
from perlin_numpy import generate_perlin_noise_2d

np.random.seed(0)

N_AGENTS = 10

GRANULARITY = 50

y = [
    generate_perlin_noise_2d((GRANULARITY, GRANULARITY), (2, 2))
    for _ in range(N_AGENTS)
]


x1 = np.linspace(0, 1 - 0.01, GRANULARITY)
x2 = np.linspace(0, 1 - 0.01, GRANULARITY)
x1, x2 = np.meshgrid(x1, x2)
X = np.stack((x1.flatten(), x2.flatten()), axis=1)
Y = y[0].flatten()

indices = np.random.choice(X.shape[0], 50, replace=False)
X = X[indices]
Y = Y[indices, None]

# ker = GPy.kern.Matern52(input_dim=2, variance=1.0, lengthscale=1.0)
# m = GPy.models.GPRegression(X, Y, ker)

# m.optimize(messages=True, max_f_eval=1000)
# m.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from scipy.stats import norm


# Define the objective function (replace with your own)
def objective_function(x):
    return np.sin(5 * x) + np.cos(10 * x) + 0.1 * x**2


# Expected Improvement acquisition function
def expected_improvement(X, gpr: GaussianProcessRegressor, best_y: float):
    mu, sigma = gpr.predict(X, return_std=True)
    sigma = np.clip(sigma, 1e-9, None)
    improvement = best_y - mu
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei.reshape(-1, 1)


# Bayesian Optimization with Active Learning
def bayesian_optimization(n_iterations, bounds):
    # Initial sample (1 random point within the bounds)
    X_samples = np.random.uniform(bounds[0], bounds[1], (1, 1))
    y_samples = objective_function(X_samples)

    for _ in range(n_iterations):
        # Fit Gaussian Process
        kernel = ConstantKernel() * RBF(
            length_scale=1.0, length_scale_bounds=(1e-1, 1e2)
        )
        gpr = GaussianProcessRegressor(kernel=kernel)
        gpr.fit(X_samples, y_samples)

        # Find the best observed value so far
        best_y = np.min(y_samples)

        # Generate candidate points (grid of 1000 points)
        X_candidates = np.linspace(bounds[0], bounds[1], 1000).reshape(-1, 1)

        # Compute Expected Improvement (EI)
        ei = expected_improvement(X_candidates, gpr, best_y)

        plot_progress(X_samples, y_samples, gpr, X_candidates, ei, bounds)

        # Select the next point to evaluate (avoid numerical edge cases)
        next_x = X_candidates[np.argmax(ei)]
        next_y = objective_function(next_x)

        # Append new sample (reshape to 2D array)
        X_samples = np.vstack([X_samples, next_x.reshape(1, -1)])
        y_samples = np.append(y_samples, next_y)

    return X_samples, y_samples


# Helper function for plotting
def plot_progress(X_samples, y_samples, gpr, X_candidates, ei, bounds):
    plt.figure(figsize=(12, 8))
    X_true = np.linspace(bounds[0], bounds[1], 100).reshape(-1, 1)
    y_true = objective_function(X_true)

    # Plot true function, GPR mean, and uncertainty
    plt.plot(X_true, y_true, "r--", label="True Function")
    mu, sigma = gpr.predict(X_true, return_std=True)
    plt.plot(X_true, mu, "b-", label="GPR Mean")
    plt.fill_between(X_true.ravel(), mu - 1.96 * sigma, mu + 1.96 * sigma, alpha=0.2)

    # Plot observed samples
    plt.scatter(X_samples, y_samples, c="k", s=100, zorder=10, label="Samples")

    # Plot EI on a twin axis
    plt.twinx()
    plt.plot(X_candidates, ei, "g--", alpha=0.5, label="Expected Improvement")
    plt.ylabel("EI", color="green")
    plt.tick_params(axis="y", labelcolor="green")

    plt.title(f"Iteration {len(X_samples) - 2}: Best y = {np.max(y_samples):.3f}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid()
    plt.show()


# Parameters
bounds = [0, 5]  # Search space bounds
n_iterations = 20  # Number of iterations

# Run Bayesian Optimization
X_samples, y_samples = bayesian_optimization(n_iterations, bounds)

# Print results
best_idx = np.argmax(y_samples)
print(f"Optimal x: {X_samples[best_idx][0]:.3f}")
print(f"Optimal y: {y_samples[best_idx]:.3f}")